In [3]:
using SparseArrays
using LinearAlgebraicRepresentation

Lar=LinearAlgebraicRepresentation
function spatial_arrangement_1(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model)

	# initializations
    fs_num = size(copFE, 1)
    rV = Array{Float64,2}(undef,0,3)
    rEV = SparseArrays.spzeros(Int8,0,0)
    rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
        for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
        end
    else
	# sequential (iterative) processing of face fragmentation
        for sigma in 1:fs_num
            #print(sigma, "/", fs_num, "\r")
            nV, nEV, nFE = Lar.Arrangement.frag_face(V, copEV, copFE, sp_idx, sigma)
			#nV, nEV, nFE = Lar.fragface(V, copEV, copFE, sp_idx, sigma)
			nV = convert(Lar.Points, nV)
            a,b,c = Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            rV=a;  rEV=b;  rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = Lar.Arrangement.merge_vertices(rV, rEV, rFE)
    return rV, rEV, rFE
end

spatial_arrangement_1 (generic function with 2 methods)

In [7]:
using SparseArrays
V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

    EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
    ])

    FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
    ])
@time spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)


  0.010088 seconds (54.52 k allocations: 3.141 MiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [9 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [10, 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [11, 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [12, 4]  =  -1
  [5 , 5]  =  -1
  [8 , 5]  =  -1
  [9 , 5]  =  1
  [5 , 6]  =  1
  [6 , 6]  =  -1
  [10, 6]  =  1
  [6 , 7]  =  1
  [7 , 7]  =  -1
  [11, 7]  =  1
  [7 , 8]  =  1
  [8 , 8]  =  1
  [12, 8]  =  1, 
  [1,  1]  =  1
  [3,  1]  =  1
  [1,  2]  =  1
  [4,  2]  =  1
  [1,  3]  =  1
  [5,  3]  =  1
  [1,  4]  =  -1
  [6,  4]  =  1
  [2,  5]  =  1
  [3,  5]  =  -1
  [2,  6]  =  1
  [4,  6]  =  -1
  [2,  7]  =  1
  [5,  7]  =  -1
  [2,  8]  =  -1
  [6,  8]  =  -1
  [3,  9]  =  -1
  [6,  9]  =  -1
  [3, 10]  =  1
  [4, 10]  =  -1
  [4, 11]  =  1
  [5, 11]  =  -1
  [5, 12]  =  1
  [6, 12]  =  1)

In [10]:
using BenchmarkTools

@btime spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)

  6.859 ms (54524 allocations: 3.14 MiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [9 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [10, 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [11, 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [12, 4]  =  -1
  [5 , 5]  =  -1
  [8 , 5]  =  -1
  [9 , 5]  =  1
  [5 , 6]  =  1
  [6 , 6]  =  -1
  [10, 6]  =  1
  [6 , 7]  =  1
  [7 , 7]  =  -1
  [11, 7]  =  1
  [7 , 8]  =  1
  [8 , 8]  =  1
  [12, 8]  =  1, 
  [1,  1]  =  1
  [3,  1]  =  1
  [1,  2]  =  1
  [4,  2]  =  1
  [1,  3]  =  1
  [5,  3]  =  1
  [1,  4]  =  -1
  [6,  4]  =  1
  [2,  5]  =  1
  [3,  5]  =  -1
  [2,  6]  =  1
  [4,  6]  =  -1
  [2,  7]  =  1
  [5,  7]  =  -1
  [2,  8]  =  -1
  [6,  8]  =  -1
  [3,  9]  =  -1
  [6,  9]  =  -1
  [3, 10]  =  1
  [4, 10]  =  -1
  [4, 11]  =  1
  [5, 11]  =  -1
  [5, 12]  =  1
  [6, 12]  =  1)